### Analyse data Titanic Disaster

In [1]:
# setup absolute path to location of package Starts and config-file 
from inspect import getsourcefile
import os.path as path, sys
current_dir = path.dirname(path.abspath(getsourcefile(lambda:0)))
sys.path.insert(0, current_dir[:current_dir.rfind(path.sep)])

# import package Starts
from Starts.startmod import *
info_mod

local_kwargs {'data_path': './data/train.csv, ./data/titanic_no_nan.csv ,./data/test.csv', 'drop_obj_col': False, 'nan_drop_col': False, 'nan_drop_row': False, 'nan_zero': False, 'nan_mean': True, 'nan_mean_neighbors': False}


{'StartML.kwargs': 'Show key words arguments from config.ini',
 'StartML.nan_columns(data)': '\n        Description: return name of all columns which have NaN_value\n\n        :param data: pandas.core.frame.DataFrame\n        :return: list of all possible NaN_column(s)\n        ',
 'StartML.nan_rows(data)': '\n        Description: return all rows containing NaN values in type DataFrame\n\n        :param data: pandas.core.frame.DataFrame\n        :param nan: Boolean-input True to search for NaN values, False for not_NaN\n        :return: data with all possible found NaN_rows or not_NaN_rows (if nan=False)\n        ',
 'StartML.pre_processing_columns(data)': '\n        Description: pre_processing columns based on information given in the config.ini\n\n        :param data: pandas.core.frame.DataFrame\n        :return: data after pre-processing\n        ',
 'StartML.pre_processing_rows(data)': '\n        Description: pre_processing rows based on information given in the config.ini\n\n     

In [2]:
train_data = idata[0]
attributes=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Dr', 'Ms', 'Capt']
train_data = StartMod.feature_engineering(train_data, old_feature='Name', new_feature='Title', new_attributes=attributes, 
                                          rm=True)

In [3]:
train_data = StartMod.feature_engineering_merge_cols(train_data, ['SibSp', 'Parch'], 'FamilySize')

In [4]:
# check NaN columns in data
train_data.head()
StartMod.nan_columns(train_data)

['Age', 'Cabin', 'Embarked', 'Title']

In [5]:
# simplify data by removing the redundant feature_columns
train_data = StartMod.feature_selection(train_data, ['Pclass', 'Ticket', 'Cabin', 'Embarked'], 
                                        dependent_label='Survived', rm=False)

Data might have NaN_value, please clean them before building model


In [6]:
# identify the index of data
StartMod.idx_columns(train_data)

[('PassengerId', 0, dtype('int64')),
 ('Survived', 1, dtype('int64')),
 ('Pclass', 2, dtype('int64')),
 ('Sex', 3, dtype('O')),
 ('Age', 4, dtype('float64')),
 ('Ticket', 5, dtype('O')),
 ('Fare', 6, dtype('float64')),
 ('Cabin', 7, dtype('O')),
 ('Embarked', 8, dtype('O')),
 ('Title', 9, dtype('O')),
 ('FamilySize', 10, dtype('int64'))]

In [7]:
# check NaN columns in data
StartMod.nan_columns(train_data)

['Age', 'Cabin', 'Embarked', 'Title']

In [8]:
# process NaN feature columns in a simple way
train_data = StartMod.process_nan_simply(train_data, nan_column='Title')

In [9]:
StartMod.nan_columns(train_data)

['Age', 'Cabin', 'Embarked']

In [10]:
# process NaN feature columns in a simple way again
train_data = StartMod.process_nan_columns(train_data)

In [11]:
# categorize feature in obj type and non_obj (numeric) type
non_obj_features, obj_features = StartMod.feature_columns(train_data)
non_obj_features, obj_features

(['PassengerId', 'Survived', 'Pclass', 'Age', 'Fare', 'FamilySize'],
 ['Sex', 'Ticket', 'Cabin', 'Embarked', 'Title'])

In [12]:
StartMod.nan_columns(train_data)

['Cabin', 'Embarked']

In [13]:
# drop column with NaN
func = lambda col: train_data.drop(col, axis=1)
train_data = func(['Cabin', 'Embarked'])
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Title,FamilySize
0,1,0,3,male,22.0,A/5 21171,7.2500,Mr,1
1,2,1,1,female,38.0,PC 17599,71.2833,Mrs,1
2,3,1,3,female,26.0,STON/O2. 3101282,7.9250,Miss,0
3,4,1,1,female,35.0,113803,53.1000,Mrs,1
4,5,0,3,male,35.0,373450,8.0500,Mr,0


In [14]:
StartMod.nan_columns(train_data)

[]

In [15]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'Ticket', 'Fare',
       'Title', 'FamilySize'],
      dtype='object')

In [16]:
# encode the columns 'Sex' and 'Title' 
train_data = StartMod.encode_label_column(train_data, label_columns=['Title', 'Sex', 'Ticket'], one_hot=False)
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Title,FamilySize
0,1,0,3,1,22.0,523,7.2500,5,1
1,2,1,1,0,38.0,596,71.2833,6,1
2,3,1,3,0,26.0,669,7.9250,4,0
3,4,1,1,0,35.0,49,53.1000,6,1
4,5,0,3,1,35.0,472,8.0500,5,0


In [17]:
train_data = train_data.drop('PassengerId', axis=1)
StartMod.idx_columns(train_data)

[('Survived', 0, dtype('int64')),
 ('Pclass', 1, dtype('int64')),
 ('Sex', 2, dtype('int64')),
 ('Age', 3, dtype('float64')),
 ('Ticket', 4, dtype('int64')),
 ('Fare', 5, dtype('float64')),
 ('Title', 6, dtype('int64')),
 ('FamilySize', 7, dtype('int64'))]

In [18]:
from sklearn.utils import shuffle
train_data = shuffle(train_data)
train_data.to_csv('./data/titanic_no_nan.csv', index=False)